<a href="https://colab.research.google.com/github/wheatfox/google_colab/blob/main/tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!ls

bert_config.json		     bert_model.ckpt.meta     tmp
bert_model.ckpt.data-00000-of-00001  chinese_L-12_H-768_A-12  tmp.py
bert_model.ckpt.index		     sample_data	      vocab.txt


In [ ]:
# test github updating
print("hello github")

In [6]:
def test_ner_on_raw_keras():
    import tensorflow as tf
    # import tensorflow.compat.v1 as v1
    import tensorflow_addons as tfa
    import numpy as np
    from pathlib import Path
    import math
    import bert as Bert
    from datetime import datetime
    import random
    random.seed(0)
    #
    _UNKNOWN_ = "un_known"

    def get_data(f):
        with Path(f).open("r", encoding="utf8")as fi:
            vocab, tags = [], []
            for line in fi.readlines():
                parts = line.strip().split(" ")
                if len(parts) != 2:
                    continue
                vocab.append(parts[0])
                tags.append(parts[1])
            vocab = list(set(vocab))
            tags = list(set(tags))
        #
        vocab2id = dict([(w, i + 1) for i, w in enumerate(vocab)])
        vocab2id.update({_UNKNOWN_: 0})
        tag2id = dict([(t, j + 1) for j, t in enumerate(tags)])
        tag2id.update({_UNKNOWN_: 0})
        return vocab2id, tag2id

    def crf_generator(f, batch_size, max_length, vocab2id, tag2id):
        data, tags, lengths = [], [], []
        count = 0
        with Path(f).open('r', encoding="utf8") as fi:
            curr_x, curr_y = [], []
            for line in fi.readlines():
                if "\n" == line:
                    if (count + 1) % batch_size == 0:
                        yield [np.array(data), np.array(lengths), np.array(tags)], np.array(tags)
                        data, lengths, tags = [], [], []
                    data.append([curr_x[i] if i < len(curr_x) else 0
                                 for i in range(max_length)])
                    tags.append([curr_y[i] if i < len(curr_y) else tag2id.get("O")
                                 for i in range(max_length)])
                    lengths.append(len(curr_x))
                    count += 1
                    curr_x, curr_y = [], []
                else:
                    parts = line.strip().split(" ")
                    if len(parts) != 2:
                        print("数据错误：{}".format(line), file=sys.stderr)
                    else:
                        curr_x.append(vocab2id.get(parts[0], 0))
                        curr_y.append(tag2id.get(parts[1]))

    class CrfGenerator(tf.keras.utils.Sequence):
        def __init__(self, f, batch_size, max_length, vocab2id, tag2id):
            # super(CrfGenerator, self).__init__()
            # inputs = self.generator_fn_from_file(f)
            data, tags, lengths = [], [], []
            with Path(f).open('r', encoding="utf8") as fi:
                curr_x, curr_y = [], []
                for line in fi.readlines():
                    if "\n" == line:
                        data.append([curr_x[i] if i < len(curr_x) else 0
                                     for i in range(max_length)])
                        tags.append([curr_y[i] if i < len(curr_y) else 0
                                     for i in range(max_length)])
                        lengths.append(len(curr_x))
                        curr_x, curr_y = [], []
                    else:
                        parts = line.strip().split(" ")
                        if len(parts) != 2:
                            print("数据错误：{}".format(line), file=sys.stderr)
                        else:
                            curr_x.append(vocab2id.get(parts[0], 0))
                            curr_y.append(tag2id.get(parts[1], 0))
            #
            self.inputs = [data, lengths, tags]
            self.batch_size = len(self.inputs[0]) if batch_size is None else batch_size

        def __len__(self):
            return math.ceil(len(self.inputs[0]) / self.batch_size)

        def __getitem__(self, idx):
            curr_data = np.array(self.inputs[0][idx * self.batch_size: (idx + 1) * self.batch_size])
            curr_lengths = np.array(self.inputs[1][idx * self.batch_size: (idx + 1) * self.batch_size])
            curr_tags = np.array(self.inputs[2][idx * self.batch_size: (idx + 1) * self.batch_size])
            # curr_lengths = np.reshape(curr_lengths, (-1, 1))
            return [curr_data, curr_lengths, curr_tags], curr_tags

        def shuffle(self):
            length = len(self.inputs[0])
            indexes = [i for i in range(length)]
            random.shuffle(indexes)
            self.inputs = [[items[j] for j in indexes] for items in self.inputs]

    class CrfLoss(tf.keras.layers.Layer):
        def __init__(self, **kwargs):
            super(CrfLoss, self).__init__(**kwargs)

        def get_config(self):
            config = {}
            base_config = super(CrfLoss, self).get_config()
            return dict(list(base_config.items()) + list(config.items()))

        def call(self, inputs=tf.keras.Input(shape=(None, None)), **kwargs):
            tags, lengths, params = kwargs.get("tags", tf.keras.Input(shape=(None,))), \
                                    kwargs.get("lengths", tf.keras.Input(shape=())), \
                                    kwargs.get("params", tf.keras.Input(shape=(None,)))
            # log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(data,
            #                                                       tags, lengths, params)
            log_likelihood, _ = tfa.text.crf_log_likelihood(inputs,
                                                            tags, lengths, params)
            loss = tf.reduce_sum(-log_likelihood)
            # loss = tf.keras.losses.sparse_categorical_crossentropy(tags, outputs, from_logits=True)
            self.add_loss(loss)
            return inputs

    class CrfMetrics(tf.keras.layers.Layer):
        def __init__(self, **kwargs):
            super(CrfMetrics, self).__init__(**kwargs)
            self.accuracy_func = tf.keras.metrics.Accuracy()

        def get_config(self):
            config = {}
            base_config = super(CrfMetrics, self).get_config()
            return dict(list(base_config.items()) + list(config.items()))

        def call(self, inputs=tf.keras.Input(shape=(None, None)), **kwargs):
            tags, lengths, params = kwargs.get("tags", tf.keras.Input(shape=(None,))), \
                                    kwargs.get("lengths", tf.keras.Input(shape=())), \
                                    kwargs.get("params", tf.keras.Input(shape=(None,)))
            lengths = tf.cast(lengths, dtype=tf.int32)
            pred_ids, _ = tfa.text.crf_decode(inputs,
                                              params,
                                              lengths)
            tags = tf.cast(tags, dtype=tf.int32)
            pred_ids = tf.cast(pred_ids, dtype=tf.int32)
            # print(tags, pred_ids, lengths)
            result = self.accuracy_func(tags, pred_ids)
            self.add_metric(result, name="accuracy")
            return inputs

    class CrfLayer(tf.keras.layers.Layer):
        def __init__(self, out_size=None, **kwargs):
            super(CrfLayer, self).__init__(**kwargs)
            v_init = tf.random_normal_initializer()
            self.crf_params = tf.Variable(initial_value=v_init(shape=(out_size, out_size), dtype=tf.float32),
                                          name="crf_params",
                                          trainable=True)
            self.accuracy_func = tf.keras.metrics.Accuracy()
            self.out_size = out_size

        def get_trans_params(self):
            return self.crf_params

        def get_config(self):
            config = {"out_size": self.out_size}
            base_config = super(CrfLayer, self).get_config()
            return dict(list(base_config.items()) + list(config.items()))

        def call(self, inputs, lengths, tags):
            # tags, lengths = kwargs.get("tags", None), kwargs.get("lengths", None)
            self.add_loss(self.compute_loss(inputs, lengths, tags))
            #
            acc = self.compute_accuracy(inputs, lengths, tags)
            self.add_metric(acc, name="accuracy")
            return inputs

        def compute_accuracy(self, inputs, lengths, tags):
            lengths = tf.cast(lengths, dtype=tf.int32)
            pred_ids = self.decode(inputs, lengths)
            tags = tf.cast(tags, dtype=tf.int32)
            pred_ids = tf.cast(pred_ids, dtype=tf.int32)
            acc = self.accuracy_func(tags, pred_ids)
            return acc

        def compute_loss(self, inputs, lengths, tags):
            log_likelihood, _ = tfa.text.crf_log_likelihood(inputs,
                                                            tags, lengths, self.crf_params)
            loss = tf.reduce_sum(-log_likelihood)
            return loss

        def decode(self, inputs, lengths):
            pred_ids, _ = tfa.text.crf_decode(inputs,
                                              self.crf_params,
                                              lengths)
            return pred_ids

    class MyCallback(tf.keras.callbacks.Callback):
        def __init__(self, model, valid_data, test_data, patience=20):
            super(MyCallback, self).__init__()
            self.top_model = model
            self.patience = patience
            self.valid = valid_data
            self.test = test_data
            #
            self.best = np.Inf
            self.best_weights = None
            self.stopped_epoch = 0
            self.waited = 0

        def on_epoch_end(self, epoch, logs=None):
            # text = "测试啊测试"
            # pred_tags = self.top_model.predict(text)
            # # print("\n" + ", ".join(["{}: {}".format(k, v) for k, v in logs.items()]))
            # print("\nepoch：{}\n文本：{}\n输出：{}".format(epoch+1, text, pred_tags))
            #
            logs = dict() if not isinstance(logs, dict) else logs
            curr_loss = logs.get("loss", 0)
            if np.less(curr_loss, self.best):
                self.waited = 0
                self.best = curr_loss
                self.top_model.save_model()
                print("\nFound better model at epoch-{}.\n".format(epoch))
                self.best_weights = self.model.get_weights()
                print("\nStarting validation...")
                self.model.evaluate(self.valid[0], self.valid[1])
                print("\nFinished validation.")
                print("\nStarting test...")
                self.model.evaluate(self.test[0], self.test[1])
                print("\nFinished test.")
            else:
                self.waited += 1
                if self.waited >= self.patience:
                    self.stopped_epoch = epoch
                    self.model.stop_training = True
                    print("\nRestore weights from best model.\n")
                    self.model.set_weights(self.best_weights)
                    self.top_model.save_model()

        def on_train_end(self, logs=None):
            if self.stopped_epoch > 0:
                print("\nEpoch %05d: early stopping.\n" % (self.stopped_epoch + 1))

    class BlstmCrf(tf.keras.Model):
        def __init__(self, params=None, **kwargs):
            super().__init__(**kwargs)
            params = dict() if params is None else params
            self.layer_name_prefix = "BlstmCrfModel_"
            max_seq_length = params.get("max_seq_len", 128)
            self.max_seq_len = max_seq_length
            vocab2id = params.get("vocab2id", None)
            if vocab2id is None:
                raise ValueError("vocab2id缺失!")
            vocab_size = len(vocab2id)
            embed_size = params.get("embed_size", 100)
            hidden_size = params.get("hidden_size", 100)
            tag2id = params.get("tag2id", None)
            if tag2id is None:
                raise ValueError("tg2id缺失!")
            out_size = len(tag2id)
            self.vocab2id = vocab2id
            self.tag2id = tag2id
            self.id2tag = dict([(i, t) for t, i in self.tag2id.items()])
            # pre_trained_model_path = params.get("pre_trained_model_path", None)
            # bert_params = Bert.params_from_pretrained_ckpt(pre_trained_model_path)
            # self.bert = Bert.BertModelLayer.from_params(bert_params, name=self.layer_name_prefix + "bert")
            #
            self.embed = tf.keras.layers.Embedding(input_dim=vocab_size,
                                                   output_dim=embed_size,
                                                   input_shape=(None, max_seq_length),
                                                   name=self.layer_name_prefix + "embedding",
                                                   dtype=tf.float32,
                                                   trainable=True)
            self.blstm = tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=hidden_size, return_sequences=True),
                name=self.layer_name_prefix + "blstm",
                dtype=tf.float32,
                trainable=True
            )
            self.dense1 = tf.keras.layers.Dense(units=out_size, name=self.layer_name_prefix + "projection",
                                                dtype=tf.float32, trainable=True)
            self.crf_layer = CrfLayer(out_size=out_size, name=self.layer_name_prefix + "crf")

        def call(self, inputs, training=None):
            x, lengths, tags = inputs
            x = self.embed(x, training=training)
            # x = self.bert(x, training=training)
            x = self.blstm(x, training=training)
            x = self.dense1(x, training=training)
            outputs = self.crf_layer(x, lengths, tags)
            return outputs

        def train_step(self, data):
            # Unpack the data. Its structure depends on your model and
            # on what you pass to `fit()`.
            x, y = data

            with tf.GradientTape() as tape:
                inputs, lengths, tags = x
                logits = self(x, training=True)  # Forward pass
                # Compute the loss value
                # (the loss function is configured in `compile()`)
                loss = self.crf_layer.compute_loss(logits, lengths, tags)

            # Compute gradients
            trainable_vars = self.trainable_variables
            gradients = tape.gradient(loss, trainable_vars)
            # Update weights
            self.optimizer.apply_gradients(zip(gradients, trainable_vars))
            # Update metrics (includes the metric that tracks the loss)
            self.compiled_metrics.update_state(y, y_pred)
            # Return a dict mapping metric names to current value
            return {m.name: m.result() for m in self.metrics}

        def decode(self, inputs, lengths):
            pred_ids = self.crf_layer.decode(inputs, lengths)
            return pred_ids

    class Ner:
        def __init__(self, params=None):
            self.layer_name_prefix = "BlstmCrfModel_"
            params = params if params is not None else dict()
            self.is_save_by_weights = params.get("is_save_by_weights", True)
            self.model_path = params.get("model_path", "model.h5")
            max_seq_length = params.get("max_seq_len", 128)
            self.max_seq_len = max_seq_length
            vocab2id = params.get("vocab2id", None)
            if vocab2id is None:
                raise ValueError("vocab2id缺失!")
            vocab_size = len(vocab2id)
            embed_size = params.get("embed_size", 100)
            hidden_size = params.get("hidden_size", 100)
            tag2id = params.get("tag2id", None)
            if tag2id is None:
                raise ValueError("tg2id缺失!")
            out_size = len(tag2id)
            self.vocab2id = vocab2id
            self.tag2id = tag2id
            self.id2tag = dict([(i, t) for t, i in self.tag2id.items()])
            #
            pre_trained_model_path = params.get("pre_trained_model_path", None)
            bert_params = Bert.params_from_pretrained_ckpt(pre_trained_model_path)
            self.embed = Bert.BertModelLayer.from_params(bert_params, name=self.layer_name_prefix + "bert")
            # self.embed = tf.keras.layers.Embedding(input_dim=vocab_size,
            #                                        output_dim=embed_size,
            #                                        input_shape=(None, max_seq_length),
            #                                        name=self.layer_name_prefix + "embedding",
            #                                        dtype=tf.float32,
            #                                        trainable=True)
            self.blstm = tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=hidden_size, return_sequences=True),
                name=self.layer_name_prefix + "blstm",
                dtype=tf.float32,
                trainable=True
            )
            self.dense1 = tf.keras.layers.Dense(units=out_size, name=self.layer_name_prefix + "projection",
                                                dtype=tf.float32, trainable=True)
            self.crf_layer = CrfLayer(out_size=out_size, name=self.layer_name_prefix + "crf")
            # v_init = tf.random_normal_initializer()
            # self.crf_params = tf.Variable(initial_value=v_init(shape=(out_size, out_size), dtype=tf.float32),
            #                               name=self.layer_name_prefix + "crf",
            #                               trainable=True)
            # self.softmax = tf.keras.layers.Softmax()
            # 创建模型
            data, lengths = tf.keras.Input(shape=(max_seq_length,), name=self.layer_name_prefix + "data"), \
                            tf.keras.Input(shape=(), name=self.layer_name_prefix + "lengths")
            tags = tf.keras.Input(shape=(max_seq_length,), name=self.layer_name_prefix + "tags")
            # outputs = tf.keras.Input(shape=(max_seq_length, out_size), name=self.layer_name_prefix + "outputs")
            # self.model = BlstmCrf(params)
            embedded = self.embed(data)
            blstm_outputs = self.blstm(embedded)
            projection_outputs = self.dense1(blstm_outputs)
            # outputs = self.softmax(projection_outputs)
            outputs = self.crf_layer(projection_outputs, lengths, tags)
            self.model = tf.keras.Model(inputs=[data, lengths, tags], outputs=outputs)

            # self.model.add_loss(tfa.text.crf_log_likelihood(projection_outputs, tags, lengths, self.crf_params)[0])
            #
            # self.model.compile(optimizer="Adam",
            #                    loss=lambda y_true, y_pred: tf.reduce_sum(
            #                        -tf.contrib.crf.crf_log_likelihood(
            #                            y_pred, tf.cast(y_true, dtype=tf.int32), lengths, self.crf_params)[0]),)
            lr_scheduler = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=0.001,
                                                                          decay_steps=10, decay_rate=0.75)
            self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler))
            # self.model.compile(optimizer="Adam", loss=crf_loss([lengths, self.crf_params]))
            self.model.summary()

        def fit(self, train_data_file, valid_data_file, test_data_file, batch_size=32, epochs=1, steps_per_epoch=None):
            train_generator = CrfGenerator(train_data_file, batch_size, self.max_seq_len, self.vocab2id, self.tag2id)
            valid_generator = CrfGenerator(valid_data_file, None, self.max_seq_len, self.vocab2id, self.tag2id)
            valid_data = valid_generator.__getitem__(0)
            test_generator = CrfGenerator(test_data_file, None, self.max_seq_len, self.vocab2id, self.tag2id)
            test_data = test_generator.__getitem__(0)
            print("获取数据, 训练/验证/测试, {}/{}/{}".format(
                len(train_generator)*batch_size, len(valid_data[-1]), len(test_data[-1])))
            #
            self.model.fit(train_generator,
                           epochs=epochs,
                           steps_per_epoch=len(data_generator) if steps_per_epoch is None else steps_per_epoch,
                           callbacks=[MyCallback(self, valid_data=valid_data, test_data=test_data, patience=2)])
            self.save_model()

        def train(self, train_file, valid_file, test_file, batch_size=32, epochs=1, patience=10):
            #
            print("{}, training started...".format(datetime.now()))
            best_value = np.Inf
            best_weights = None
            wait = 0
            # 自定义训练过程
            data_generator = CrfGenerator(train_file, batch_size, self.max_seq_len, self.vocab2id, self.tag2id)
            for epoch in range(epochs):
                steps = len(data_generator)
                loss_sum = 0
                acc_sum = 0
                for step in range(steps):
                    x, y = data_generator.__getitem__(step)
                    inputs, lengths, tags = x
                    with tf.GradientTape() as tape:
                        logits = self.model(x, training=True)
                        loss = self.crf_layer.compute_loss(logits, lengths, tags)
                        acc = self.crf_layer.compute_accuracy(logits, lengths, tags)
                        loss_sum += loss
                        acc_sum += acc
                    train_vars = [v for layer in [self.blstm, self.dense1, self.crf_layer]
                                  for v in layer.trainable_variables]
                    grads = tape.gradient(loss, train_vars)
                    # print(type(self.model.trainable_variables))
                    # Run one step of gradient descent by updating
                    # the value of the variables to minimize the loss.
                    self.model.optimizer.apply_gradients(zip(grads, train_vars))
                    print("{}, epoch: {}/{}, step: {}/{}, loss: {:.3f}, acc: {:.3f}".format(
                        datetime.now(), epoch+1, epochs, step+1, steps, loss, acc))
                #
                loss_avg = loss_sum / steps
                acc_avg = acc_sum / steps
                if np.less(loss_avg, best_value):
                    best_value = loss_avg
                    best_weights = self.model.get_weights()
                    self.save_model()
                    print("{}, found better model at epoch-{}. avg loss: {:.3f}, avg acc: {:.3f}".format(
                        datetime.now(), epoch, loss_avg, acc_avg
                    ))
                    #
                    print("{}, validation started...".format(datetime.now()))
                    valid_generator = CrfGenerator(valid_file, None, self.max_seq_len, self.vocab2id, self.tag2id)
                    for step in range(len(valid_generator)):
                        x, y = valid_generator.__getitem__(step)
                        x = [xx[0:10] for xx in x]
                        y = y[0:10]
                        inputs, lengths, tags = x
                        logits = self.model(x, training=False)
                        loss = self.crf_layer.compute_loss(logits, lengths, tags)
                        acc = self.crf_layer.compute_accuracy(logits, lengths, tags)
                        print("{}, validation, loss: {:.3f}, acc: {:.3f}".format(
                            datetime.now(), loss, acc
                        ))
                    print("{}, validation done.".format(datetime.now()))
                    print("{}, test started...".format(datetime.now()))
                    test_generator = CrfGenerator(test_file, None, self.max_seq_len, self.vocab2id, self.tag2id)
                    for step in range(len(test_generator)):
                        x, y = test_generator.__getitem__(step)
                        x = [xx[0:10] for xx in x]
                        y = y[0:10]
                        inputs, lengths, tags = x
                        logits = self.model(x, training=False)
                        loss = self.crf_layer.compute_loss(logits, lengths, tags)
                        acc = self.crf_layer.compute_accuracy(logits, lengths, tags)
                        print("{}, test, loss: {:.3f}, acc: {:.3f}".format(
                            datetime.now(), loss, acc
                        ))
                    print("{}, test done.".format(datetime.now()))
                else:
                    wait += 1
                    if wait >= patience:
                        self.model.set_weights(best_weights)
                        self.save_model()
                        print("{}, early stopped at epoch-{}".format(
                            datetime.now(), epoch
                        ))
                # 每个epoch结束后都随机改变下训练数据的顺序
                data_generator.shuffle()
            print("{}, training done.".format(datetime.now()))

        def save_model(self, model_path=None):
            if model_path is None:
                model_path = self.model_path
            # with self.graph.as_default():
            if self.is_save_by_weights:
                self.model.save_weights(model_path)
            else:
                self.model.save(model_path)

        def load_model(self, model_path=None):
            if model_path is None:
                model_path = self.model_path
            # with self.graph.as_default():
            if self.is_save_by_weights:
                self.model.load_weights(model_path)
            else:
                self.model = tf.keras.models.load_model(
                    model_path, custom_objects={"CrfLayer": CrfLayer})

        def predict(self, text):
            text_length = len(text)
            words = np.array([[self.vocab2id.get(text[i] if i < len(text) else "unknown", 0)
                               for i in range(self.max_seq_len)]])
            tags = np.array([[self.tag2id.get('O') for _ in range(self.max_seq_len)]])
            lengths = np.array([len(words[0])])
            # with self.graph.as_default():
            outputs = self.model([words, lengths, tags])
            pred_ids = self.crf_layer.decode(outputs, lengths)
            # print(pred_ids)
            pred_ids = pred_ids.numpy().tolist()[0]
            pred_tags = [self.id2tag.get(_id, "O") for _id in pred_ids[0: text_length]]
            return pred_tags

    # ------------------------------------------------
    # import torch
    #
    # class BlstmCrfOnTorch(torch.nn):
    #     def __init__(self):
    #         super(BlstmCrfOnTorch, self).__init__()

    #
    f = "tmp/train.txt"
    vocab2id, tag2id = get_data(f)
    params = {"vocab2id": vocab2id, "hidden_size": 200, "embed_size": 1000,
              "tag2id": tag2id, "batch_size": 32, "max_seq_len": 64,
              "is_save_by_weights": True, "model_path": "model.h5",
              "pre_trained_model_path": "tmp\\chinese_L-12_H-768_A-12"}
    batch_size = params.get("batch_size")

    #
    class ToyModel:
        def __init__(self, kwargs):
            self.layer_name_prefix = "ToyModel_"
            #
            kwargs = kwargs if kwargs is not None else dict()
            self.is_save_by_weights = kwargs.get("is_save_by_weights", True)
            self.model_path = kwargs.get("model_path", "model.h5")
            max_seq_length = kwargs.get("max_seq_len", 128)
            self.max_seq_len = max_seq_length
            vocab2id = kwargs.get("vocab2id", None)
            if vocab2id is None:
                raise ValueError("vocab2id缺失!")
            vocab_size = len(vocab2id)
            embed_size = kwargs.get("embed_size", 100)
            hidden_size = kwargs.get("hidden_size", 100)
            tag2id = kwargs.get("tag2id", None)
            if tag2id is None:
                raise ValueError("tg2id缺失!")
            out_size = len(tag2id)
            self.vocab_size = vocab_size
            self.out_size = out_size
            self.vocab2id = vocab2id
            self.tag2id = tag2id
            self.id2tag = dict([(i, t) for t, i in self.tag2id.items()])
            #
            embedding = tf.keras.layers.Embedding(
                input_dim=vocab_size,
                output_dim=embed_size,
                input_shape=(None, max_seq_length),
                name=self.layer_name_prefix + "embedding",
                dtype=tf.float32,
                trainable=True)

            dense1 = tf.keras.layers.Dense(out_size, activation="relu", dtype=tf.float32,
                                           name=self.layer_name_prefix + "dense1")
            softmax = tf.keras.layers.Softmax(name=self.layer_name_prefix + "softmax")
            inputs = tf.keras.Input(shape=(self.max_seq_len,))
            outputs = embedding(inputs)
            outputs = dense1(outputs)
            outputs = tf.reduce_sum(outputs, axis=1)
            outputs = softmax(outputs)
            model2 = tf.keras.Model(inputs, outputs)
            model2.compile(optimizer="Adam", loss=tf.keras.losses.sparse_categorical_crossentropy)
            model2.summary()
            self.model = model2

        def fit(self):
            batch_size = 32
            self.model.fit(
                x=np.random.randint(0, self.vocab_size, (4096, self.max_seq_len)),
                y=np.random.randint(0, self.out_size, (4096,)),
                epochs=3,
                batch_size=batch_size,
                steps_per_epoch=math.ceil(4096/batch_size))
            self.model.save("model_b.h5")

        def load_model(self, model_path):
            self.model = tf.keras.models.load_model(model_path)

        def predict(self, text):
            text_length = len(text)
            words = np.array([[self.vocab2id.get(text[i] if i < len(text) else "unknown", 0)
                               for i in range(self.max_seq_len)]])
            # with self.graph.as_default():
            outputs = self.model(words)
            pred_ids = np.argmax(outputs, axis=-1).tolist()
            return pred_ids

    class TestTwoKerasModelAtTheSameTime:
        def __init__(self):
            self.model_a = Ner(kwargs)
            self.model_a.load_model(model_path="model_a.h5")
            self.model_b = ToyModel(kwargs)
            self.model_b.load_model("model_b.h5")

        def process(self, text):
            result_a = self.model_a.predict(text)
            result_b = self.model_b.predict(text)
            print("result by model a: {}".format(result_a))
            print("result by model b: {}".format(result_b))

    dev_f = "tmp/dev.txt"
    test_f = "tmp/test.txt"
    # model = ToyModel(kwargs)
    # model.fit()
    model = Ner(params)
    # model.fit(f, dev_f, test_f, batch_size=batch_size, epochs=1, steps_per_epoch=3)
    model.train(f, dev_f, test_f, batch_size=batch_size, epochs=5)
    model.load_model("model.h5")
    tags = model.predict("雅诗兰黛防晒霜好用吗")
    print(tags)
    #
    # handler = TestTwoKerasModelAtTheSameTime()
    # while True:
    #     t = input("请输入: ")
    #     handler.process(t)
print("加载完毕.")

加载完毕.
